In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Input, Dense, MaxPooling2D, Conv2D, Flatten, concatenate
from keras.optimizers import Adam
from keras import regularizers
from keras import optimizers
from keras import initializers
from keras.preprocessing import image
from keras.models import Model

import keras.backend as K

from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Read the train dataset
trainfile = pd.read_csv("train.csv")

# Get the input data and label
xtrain = trainfile.iloc[:,1:].values.astype('float32')
ytrain = trainfile.iloc[:,0].values.astype('int')

# Rescale input
xtrain = xtrain/255

# Get the labels as one hot encoding arrays
enc = OneHotEncoder(n_values=10)
enc.fit(ytrain)
labels = enc.transform(ytrain).toarray()

# Reshape
xtrain = xtrain.reshape(xtrain.shape[0], 28, 28, 1)
ylabels = labels.reshape(ytrain.shape[0], 10)

In [ ]:
# Define the model
def model(Xshape):
    
    input_img = Input(shape=Xshape)

    tower_1 = Conv2D(16, (3, 3), padding='same', activation='relu')(input_img)
    tower_1 = MaxPooling2D(pool_size=(2, 2))(tower_1)
    tower_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(tower_1)
    tower_1 = MaxPooling2D(pool_size=(2, 2))(tower_1)
    
    tower_2 = Conv2D(16, (5, 5), padding='same', activation='relu')(input_img)
    tower_2 = MaxPooling2D(pool_size=(2, 2))(tower_2)
    tower_2 = Conv2D(32, (5, 5), padding='same', activation='relu')(tower_2)
    tower_2 = MaxPooling2D(pool_size=(2, 2))(tower_2)

    output = concatenate([tower_1, tower_2], axis=1)
    
    output = Flatten()(output)
    output = Dense(256, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01))(output)
    output = Dense(128, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01))(output)
    output = Dense(64, activation='relu', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0.01))(output)
    output = Dense(10, activation='softmax', kernel_initializer='glorot_normal')(output)
    
    model = Model(inputs = input_img, outputs = output)

    return model

In [ ]:
# Compile the model 
mymodel = model(xtrain.shape[1:])
opt = optimizers.Adam(lr=0.01, decay=0.0002)

mymodel.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# Do some image augmentation if wanted
gen = image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1)

mymodel.fit_generator(gen.flow(xtrain, ylabels, batch_size=128), epochs = 20, steps_per_epoch=len(xtrain)/128)

In [ ]:
# First train with small mini batch size and default learning rate
#mymodel.fit(xtrain, ylabels, epochs = 30, batch_size=64)
# Then train with smaller learning rate and larger batch size
#K.set_value(mymodel.optimizer.lr, 0.001)
#mymodel.fit(xtrain, ylabels, epochs = 10, batch_size=512)
#K.set_value(mymodel.optimizer.lr, 0.0001)
mymodel.fit(xtrain, ylabels, epochs = 5, batch_size=1024)


In [ ]:
# Read the test file and preprocess it
testfile = pd.read_csv("test.csv")
xtest = testfile.values.astype('float32')

xtest = xtest/255

xtest = xtest.reshape(xtest.shape[0], 28, 28, 1)

In [ ]:
# Make the predictions and save them to a file
predictions = mymodel.predict(xtest)
predictions = np.argmax(predictions, axis=1)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("predsInception.csv", index=False, header=True)